### This notebook is dervied from [Slawek Biels](https://www.kaggle.com/slawekbiel/positive-score-with-detectron-2-3-training) introduction to Detectron and also had some inspiration by [Ammar Alhaj Ali](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-training). The goal is to add simple and flexible augmentation and to keep track of all with Weights & Biases.

In [1]:
#%pip install torch torchvision torchaudio

In [2]:
#%pip install opencv-python-headless ## "headless" because otherwise 'sudo apt-install' is required but not possible with sagemaker studiolab permissions

In [3]:
#%pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [4]:
#%pip install wandb --upgrade

### Free space of deleted models in sagemaker

In [5]:
!rm -rf /home/studio-lab-user/.local/share/Trash/

### Imports

In [6]:
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt 
import numpy as np

import pycocotools.mask as mask_util

import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import detection_utils as utils
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.config import CfgNode
from detectron2.modeling import build_model
from detectron2.utils import comm

import onecycle.onecycle as onecycle

import wandb
from pprint import pprint
import copy
import torch
import shutil
setup_logger()

<Logger detectron2 (DEBUG)>

### Set environment variable for wandb to work with sagemaker:

In [7]:
os.environ['WANDB_NOTEBOOK_NAME'] = './Detectron-wandb.ipynb'

### Both Detectron and WandB use configs. To help keeping track of everything, use a single dict that both can dervie parameters from

In [8]:
def build_run_config(project_name, 
                     project_params, 
                     sweep = False, 
                     epochs = 5, 
                     lr = 4e-4, 
                     wd = 1e-4, 
                     batch_size = 5, 
                     roi_bs = 512, 
                     mixed_precision = False, 
                     from_pretrain = False, 
                     pretrain_path = 'models/pretrain.pth', 
                     resume = False
                    ):
    params = project_params[project_name]
    
    if from_pretrain:
        assert(Path(pretrain_path).exists()), f'Pretrained model: "{pretrain_path}" doesnt exist.'
    
    run_config = {
        'project': dict(
            name = project_name,
            train_name = project_name + '_train',
            val_name = project_name + '_val',
            train_annotations = params['annotation_paths']['train'],
            val_annotations = params['annotation_paths']['val'],
            content_path = params['content_path'],
            sweep = sweep,
        ),
        ## hyper dict is replaced by sweep_config if wandb sweep is used
        'hyper': dict(
            epochs = epochs,
            learning_rate = lr,
            batch_size = batch_size,
            roi_bs = roi_bs,
            weight_decay = wd,
            mixed_precision = mixed_precision,
            nesterov = False,
            contrast = 0.3,
            brightness = 0.3,
            saturation = 0.3,
            rotation = 15,
            crop_size = False,
            shortest_edge_size = (440, 480, 520, 560, 580, 620),
            hor_vert_flips = (True, True),
        ),
        'model': dict(
            arch = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml",
            #arch = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml",
            #arch = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml",
            classes = params['classes'],
            from_pretrain = from_pretrain,
            pretrain_path = pretrain_path,
            resume = resume
        ),
        'sweep_config': dict(
            method = 'random',
            metric = {
                'name': 'MaP IoU',
                'goal': 'maximize',
            },
            parameters = {
                'epochs': {
                    'value': 5,
                },
                'batch_size':{
                    'values': [2, 4, 8],
                },
                'roi_bs':{
                    'values': [64, 128, 256, 512],
                },
                'learning_rate': {
                    #'distribution': 'uniform',
                    #'min': 3e-4, ## 5e-5
                    #'max': 5e-4,
                    'value': lr,
                },
                'weight_decay': {
                    'distribution': 'uniform',
                    'min': 1e-5,
                    'max': 1e-3,
                    #'values': [1e-3,1e-4,1e-5],
                    #'value': 1e-4,
                },
                'shortest_edge_size': {
                    #'values': [224, 440, 520, 640, 768, 800]
                    'value': (440, 480, 520, 560, 580, 620)
                },
                'hor_vert_flips': {
                    #'values': [(True, False), (False, True), (True, True), (False, False)]
                    'value': (True, True)
                },
                'contrast':{
                    #'distribution': 'uniform',
                    #'min': 0.05,
                    #'max': 0.5,
                    'value': 0.3
                },
                'brightness':{
                    'value': 0.3,
                    #'distribution': 'uniform',
                    #'min': 0.05,
                    #'max': 0.5,
                },
                'saturation':{
                    'value': False,
                    #'distribution': 'uniform',
                    #'min': 0.05,
                    #'max': 0.5,
                },
                'rotation': {
                    'value': 15,
                    #'distribution': 'uniform',
                    #'min': 5,
                    #'max': 45,
                },
                'crop_size':{
                    'value': False, #[(224, 303),(300, 406), (448, 606),]
                },
                'mixed_precision':{
                    'values': [True, False],
                },
                'nesterov': {
                    'values': [True, False],
                },
            }
        )
    }
    
    return run_config

### Build a detectron config from parameter dict
See the docs [detectron2.config](https://detectron2.readthedocs.io/en/latest/modules/config.html#yaml-config-references) for details and additional parameters. Add nodes for 1cycle policy and Augmentations to be used in the default setup of the Trainer and DataLoader.

In [9]:
def build_train_config(run_cfg, pretrain = False, pretrain_path = None, resize_val = False, min_size_train = None, max_size_train = None):
    cfg = get_cfg() ## Build detectron config to add to / change values
    
    ## Model
    cfg.merge_from_file(model_zoo.get_config_file(run_cfg['model']['arch']))
    if pretrain:
        assert(Path(pretrain_path).exists()), f'Pretrained model: "{pretrain_path}" doesnt exist.'
        cfg.MODEL.WEIGHTS = pretrain_path
    else:
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(run_cfg['model']['arch'])  # Let training initialize from model zoo
        
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = run_cfg['model']['classes']
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
    
    ## Dataloaders
    cfg.DATASETS.TRAIN = (run_cfg['project']['train_name'],)
    cfg.DATASETS.TEST = (run_cfg['project']['val_name'],)
    cfg.INPUT.MASK_FORMAT = 'bitmask'
    cfg.DATALOADER.NUM_WORKERS = 0

    ## Batch sizes
    cfg.SOLVER.IMS_PER_BATCH = run_cfg['hyper']['batch_size']
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = run_cfg['hyper']['roi_bs']

    cfg.SOLVER.BASE_LR = run_cfg['hyper']['learning_rate']     
    
    ## Training Phase
    iters_per_epoch = len(DatasetCatalog.get(run_cfg['project']['train_name'])) // run_cfg['hyper']['batch_size']
    cfg.SOLVER.MAX_ITER = run_cfg['hyper']['epochs'] * iters_per_epoch
    print(f'Total iterations: {cfg.SOLVER.MAX_ITER}')
    cfg.SOLVER.WEIGHT_DECAY = run_cfg['hyper']['weight_decay']
    cfg.SOLVER.NESTEROV = run_cfg['hyper']['nesterov']
    cfg.SOLVER.STEPS = []
    ## 1cycle policy
    onecycle_dict = {
        'LR_MAX': run_cfg['hyper']['learning_rate'],
        'DIV': 25,
        'DIV_FINAL': 1e4,
        'PCT': 0.3,
        'MOMS': (0.95, 0.85, 0.95)
    }
    cfg.ONECYCLE = CfgNode(onecycle_dict)
        
    ## Augmentatinos 
    hor_flip, vert_flip = run_config['hyper']['hor_vert_flips']
    augs_dict = {
        'hor_flip':           hor_flip,
        'vert_flip':          vert_flip,
        'contrast':           run_cfg['hyper']['contrast'],
        'brightness':         run_cfg['hyper']['brightness'],
        'saturation':         run_cfg['hyper']['saturation'],
        'rotation':           run_cfg['hyper']['rotation'],
        'crop_size':          run_cfg['hyper']['crop_size'],
        'shortest_edge_size': run_cfg['hyper']['shortest_edge_size']
    }
    cfg.AUGS = CfgNode(augs_dict) 

    ## Validation Phase
    cfg.TEST.EVAL_PERIOD = 500
    cfg.SOLVER.CHECKPOINT_PERIOD = 500
    ## Turn off resize for validation
    ## Make shure trainer has seen original image size
    if not resize_val:
        cfg.INPUT.MIN_SIZE_TEST = 0 

    ## Mixed Precision
    if run_config['hyper']['mixed_precision']:
        cfg.SOLVER.AMP.ENABLED = True
        
    return cfg

### Project Parameters
to easily change between pre & acctual task training

In [10]:
project_params = {
    'CellSeg': {
        'content_path': Path('SartoriusCellSeg'),
        'annotation_paths': {'train': 'SartoriusCellSeg/annotations_train.json',
                             'val': 'SartoriusCellSeg/annotations_val.json'
                            },
        'classes': 3,
        #'eval_mult': 4,
        
    },
    'LiveCell': {
        'content_path': Path('LiveCell/png'),
        'annotation_paths': {'train': 'LiveCell/livecell_coco_train_encoded.json',
                             'val': 'LiveCell/livecell_coco_val_encoded.json'
                            },
        'classes': 8,
        #'eval_mult': 0.25,
        
    }
}

In [11]:
run_config = build_run_config('CellSeg', project_params, sweep = True, epochs = 25, from_pretrain = False, pretrain_path ='models/model_XRes_Small50eposCycle.pth' , resume = False, batch_size = 4, roi_bs = 128, lr = 3e-3)
pprint(run_config)

{'hyper': {'batch_size': 4,
           'brightness': 0.3,
           'contrast': 0.3,
           'crop_size': False,
           'epochs': 25,
           'hor_vert_flips': (True, True),
           'learning_rate': 0.003,
           'mixed_precision': False,
           'nesterov': False,
           'roi_bs': 128,
           'rotation': 15,
           'saturation': 0.3,
           'shortest_edge_size': (440, 480, 520, 560, 580, 620),
           'weight_decay': 0.0001},
 'model': {'arch': 'COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml',
           'classes': 3,
           'from_pretrain': False,
           'pretrain_path': 'models/model_XRes_Small50eposCycle.pth',
           'resume': False},
 'project': {'content_path': PosixPath('SartoriusCellSeg'),
             'name': 'CellSeg',
             'sweep': True,
             'train_annotations': 'SartoriusCellSeg/annotations_train.json',
             'train_name': 'CellSeg_train',
             'val_annotations': 'SartoriusCellSeg/anno

In [12]:
register_coco_instances(run_config['project']['train_name'],{}, run_config['project']['train_annotations'], run_config['project']['content_path'])
register_coco_instances(run_config['project']['val_name'],{}, run_config['project']['val_annotations'], run_config['project']['content_path'])

## LR Finder
Uncomment to use the learning rate finder. Best to comment out, set learning rate in config and rerun notebook to free memory.

In [13]:
#cfg = build_train_config(run_config, mixed_precision=True)
#os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)
#trainer = onecycle.CycleTrainer(cfg)
#lrs = trainer.lr_find(end_lr = 0.1)

### Result of lr_find: 
* XRes 101 | Small | CellSeg: 1e-3
* XRes 101 | Full | CellSeg: 5e-3
* Resnet 50 | Full | CellSeg: 5e-3
* Resnet 50 | Full | CellSeg | fp16: 3e-3

### Perform training with previously set parameters and wandb to record

In [14]:
def train(cfg = None):   
    
    with wandb.init(project = run_config['project']['name'], sync_tensorboard = True, name = 'R50-oneCycle-sweeps'):
        
        if run_config['project']['sweep']:
            run_config['hyper'] = wandb.config

        cfg = build_train_config(run_config, pretrain = run_config['model']['from_pretrain'], pretrain_path= run_config['model']['pretrain_path'])

        if not run_config['model']['resume']:
            ## Clear the output folder if training doesn't continue earlier training
            if os.path.isdir(cfg.OUTPUT_DIR):
                shutil.rmtree(cfg.OUTPUT_DIR)
            os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
        trainer = onecycle.CycleTrainer(cfg)
        #trainer = onecycle.AugTrainer(cfg)
        trainer.resume_or_load(resume=run_config['model']['resume'])

        trainer.train()

In [ ]:
#sweep_id = wandb.sweep(run_config['sweep_config'], project = run_config['project']['name'])
sweep_id = 'ben-karr/CellSeg/gzjsnknx'
wandb.agent(sweep_id, train, count = 8)

wandb: Agent Starting Run: qwchocc2 with config:
wandb: 	batch_size: 4
wandb: 	brightness: 0.3
wandb: 	contrast: 0.3
wandb: 	crop_size: False
wandb: 	epochs: 5
wandb: 	hor_vert_flips: [True, True]
wandb: 	learning_rate: 0.003
wandb: 	mixed_precision: True
wandb: 	nesterov: False
wandb: 	roi_bs: 512
wandb: 	rotation: 15
wandb: 	saturation: False
wandb: 	shortest_edge_size: [440, 480, 520, 560, 580, 620]
wandb: 	weight_decay: 8.546613308981868e-05
wandb: Currently logged in as: ben-karr (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


[01/06 15:28:20 d2.data.datasets.coco]: Loading SartoriusCellSeg/annotations_train.json takes 1.31 seconds.
[01/06 15:28:20 d2.data.datasets.coco]: Loaded 485 images in COCO format from SartoriusCellSeg/annotations_train.json
Total iterations: 605
[01/06 15:28:28 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [RandomRotation(angle=[-15, 15], expand=False), ResizeShortestEdge(short_edge_length=[440, 480, 520, 560, 580, 620], max_size=1333, sample_style='choice'), RandomFlip(prob=0.5), RandomFlip(prob=0.5, horizontal=False, vertical=True), RandomContrast(intensity_min=0.7, intensity_max=1.3), RandomBrightness(intensity_min=0.7, intensity_max=1.3)]
[01/06 15:28:29 d2.data.datasets.coco]: Loading SartoriusCellSeg/annotations_train.json takes 1.02 seconds.
[01/06 15:28:29 d2.data.datasets.coco]: Loaded 485 images in COCO format from SartoriusCellSeg/annotations_train.json
[01/06 15:28:29 d2.data.build]: Removed 0 images with no usable annotations. 485 images left.


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

[01/06 15:28:30 d2.engine.train_loop]: Starting training from iteration 0


/home/studio-lab-user/.conda/envs/test/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/studio-lab-user/SartoriusCellSegmentation/onecycle.py:130: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

[01/06 16:16:14 d2.data.datasets.coco]: Loading SartoriusCellSeg/annotations_train.json takes 1.19 seconds.
[01/06 16:16:14 d2.data.datasets.coco]: Loaded 485 images in COCO format from SartoriusCellSeg/annotations_train.json
Total iterations: 605
[01/06 16:16:16 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [RandomRotation(angle=[-15, 15], expand=False), ResizeShortestEdge(short_edge_length=[440, 480, 520, 560, 580, 620], max_size=1333, sample_style='choice'), RandomFlip(prob=0.5), RandomFlip(prob=0.5, horizontal=False, vertical=True), RandomContrast(intensity_min=0.7, intensity_max=1.3), RandomBrightness(intensity_min=0.7, intensity_max=1.3)]
[01/06 16:16:17 d2.data.datasets.coco]: Loading SartoriusCellSeg/annotations_train.json takes 1.02 seconds.
[01/06 16:16:17 d2.data.datasets.coco]: Loaded 485 images in COCO format from SartoriusCellSeg/annotations_train.json
[01/06 16:16:18 d2.data.build]: Removed 0 images with no usable annotations. 485 images left.


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

[01/06 16:16:18 d2.engine.train_loop]: Starting training from iteration 0


/home/studio-lab-user/SartoriusCellSegmentation/onecycle.py:130: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


[01/06 16:17:06 d2.utils.events]:  eta: 0:22:21  iter: 19  total_loss: 3.005  loss_cls: 1.004  loss_box_reg: 0.3534  loss_mask: 0.6862  loss_rpn_cls: 0.5073  loss_rpn_loc: 0.447  time: 2.5792  data_time: 2.0153  lr: 0.00019717  max_mem: 4256M
[01/06 16:17:46 d2.utils.events]:  eta: 0:20:14  iter: 39  total_loss: 2.657  loss_cls: 0.7474  loss_box_reg: 0.5481  loss_mask: 0.6322  loss_rpn_cls: 0.3327  loss_rpn_loc: 0.3677  time: 2.2741  data_time: 1.6167  lr: 0.00043583  max_mem: 4256M
[01/06 16:18:24 d2.utils.events]:  eta: 0:18:29  iter: 59  total_loss: 2.457  loss_cls: 0.6837  loss_box_reg: 0.6425  loss_mask: 0.488  loss_rpn_cls: 0.2514  loss_rpn_loc: 0.3556  time: 2.1377  data_time: 1.5097  lr: 0.00080787  max_mem: 4256M
[01/06 16:19:14 d2.utils.events]:  eta: 0:18:21  iter: 79  total_loss: 2.217  loss_cls: 0.5923  loss_box_reg: 0.6108  loss_mask: 0.4018  loss_rpn_cls: 0.2502  loss_rpn_loc: 0.3463  time: 2.2305  data_time: 2.0942  lr: 0.0012691  max_mem: 4256M
[01/06 16:19:55 d2.utils